# Generator based Coroutine

## basic behavior

In [1]:
def simple_coroutine(): 
    print('-> coroutine started')
    x = yield
    print('-> coroutine received:', x)

In [2]:
my_coro = simple_coroutine()

In [3]:
my_coro

<generator object simple_coroutine at 0x1093ef030>

In [4]:
type(my_coro)

generator

In [5]:
next(my_coro)

-> coroutine started


In [6]:
my_coro.send(30)

-> coroutine received: 30


StopIteration: 

## Coroutine (actually generator) Status (Life cycle)

In [12]:
def simple_coro2(a): 
    print('-> Started: a =', a)
    b= yield a
    print('-> Received: b =', b)
    c = yield a+b
    print('-> Received: c =', c)

In [13]:
my_coro2 = simple_coro2(14)

In [14]:
from inspect import getgeneratorstate

In [15]:
getgeneratorstate(my_coro2)

'GEN_CREATED'

In [16]:
my_coro2.send(12)

TypeError: can't send non-None value to a just-started generator

In [17]:
next(my_coro2)

-> Started: a = 14


14

In [18]:
getgeneratorstate(my_coro2)

'GEN_SUSPENDED'

In [19]:
my_coro2.send(50)

-> Received: b = 50


64

In [20]:
getgeneratorstate(my_coro2)

'GEN_SUSPENDED'

In [21]:
my_coro2.send(1050)

-> Received: c = 1050


StopIteration: 

In [22]:
getgeneratorstate(my_coro2)

'GEN_CLOSED'

* 코루틴 동작 방식 (그림 출처 : fluent python(2015))

![gen_coro_flow](./images/gen_coro_flow.png)

## Sample : 이동평균을 계산하는 코루틴

In [23]:
def averager():
    total = 0.0
    count = 0
    average = None 
    while True:
        term = yield average 
        total += term
        count += 1
        average = total/count


In [24]:
coro_avg = averager()
next(coro_avg)

In [25]:
getgeneratorstate(coro_avg)

'GEN_SUSPENDED'

In [26]:
coro_avg.send(10)

10.0

In [27]:
coro_avg.send(20)

15.0

In [28]:
coro_avg.send(40)

23.333333333333332

In [29]:
coro_avg.send(None)

TypeError: unsupported operand type(s) for +=: 'float' and 'NoneType'

In [30]:
coro_avg.send(30)

StopIteration: 

In [31]:
getgeneratorstate(coro_avg)

'GEN_CLOSED'

## 코루틴의 종료와 예외처리

In [37]:
coro_avg = averager()
next(coro_avg)
getgeneratorstate(coro_avg)

'GEN_SUSPENDED'

In [38]:
coro_avg.send(50)

50.0

In [39]:
coro_avg.send(120)

85.0

In [40]:
coro_avg.close()

In [41]:
getgeneratorstate(coro_avg)

'GEN_CLOSED'

### 코루틴 예외처리를 방법을 설명하기 위한 제너레이터

In [44]:
class DemoException(Exception):
    """An exception type for the demonstration."""

def demo_exc_handling(): 
    print('-> coroutine started') 
    while True:
        try:
            x = yield
        except DemoException:
            print('*** DemoException handled. Continuing...')
        else:
            print('-> coroutine received: {!r}'.format(x))
    raise RuntimeError('This line should never run.')

In [45]:
exc_coro = demo_exc_handling()
next(exc_coro)
getgeneratorstate(exc_coro)

-> coroutine started


'GEN_SUSPENDED'

In [46]:
exc_coro.send(11)

-> coroutine received: 11


In [48]:
exc_coro.send(20)

-> coroutine received: 20


In [50]:
exc_coro.close()

In [51]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

#### 예외 발생시키기

In [52]:
exc_coro = demo_exc_handling()
next(exc_coro)
getgeneratorstate(exc_coro)

-> coroutine started


'GEN_SUSPENDED'

In [53]:
exc_coro.throw(DemoException)

*** DemoException handled. Continuing...


In [54]:
getgeneratorstate(exc_coro)

'GEN_SUSPENDED'

In [55]:
exc_coro.throw(ZeroDivisionError)

ZeroDivisionError: 

In [56]:
getgeneratorstate(exc_coro)

'GEN_CLOSED'

#### 코루틴 종료시 어떤작업을 실행하려면 try-finally 구문 사용

In [58]:
class DemoException(Exception):
    """An exception type for the demonstration."""

def demo_finally():
    print('-> coroutine started') 
    try:
        while True: 
            try:
                x = yield
            except DemoException:
                print('*** DemoException handled. Continuing...') 
            else:
                print('-> coroutine received: {!r}'.format(x))
    finally:
        print('-> coroutine ending')

In [59]:
exc_coro = demo_finally()
next(exc_coro)
getgeneratorstate(exc_coro)

-> coroutine started


'GEN_SUSPENDED'

In [60]:
exc_coro.send(11)

-> coroutine received: 11


In [61]:
exc_coro.throw(DemoException)

*** DemoException handled. Continuing...


In [62]:
exc_coro.throw(ZeroDivisionError)

-> coroutine ending


ZeroDivisionError: 

## 코루틴에서 값을 반환하기

In [107]:
from collections import namedtuple
Result = namedtuple('Result', 'count average')

def averager():
    total = 0.0
    count = 0
    average = None 
    while True:
        term = yield average 
        if term is None:
            break
        
        total += term
        count += 1
        average = total/count
    return Result(count, average)


In [108]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)
coro_avg.send(None)

StopIteration: Result(count=3, average=15.5)

In [65]:
coro_avg = averager()
next(coro_avg)
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)

try:
    coro_avg.send(None)
except StopIteration as exc:
    result = exc.value
print(result)

Result(count=3, average=15.5)


In [70]:
coro_avg = averager()
next(coro_avg)

In [71]:
coro_avg.send(10)
coro_avg.send(30)
coro_avg.send(6.5)

15.5

In [72]:
r = coro_avg.close()

In [73]:
print(r)

None


## yield from

In [74]:
def gen():
    for c in 'AB':
        yield c
    for i in range(1,3):
        yield i


In [75]:
gen()

<generator object gen at 0x112711d20>

In [76]:
list(gen())

['A', 'B', 1, 2]

In [77]:
def gen():
    yield from 'AB'
    yield from range(1,3)


In [79]:
gen()

<generator object gen at 0x112d9cfb0>

In [80]:
list(gen())

['A', 'B', 1, 2]

In [81]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)

    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))
        
    def __repr__(self):
        return f"Senetence ({reprlib.repr(self.text)})"

In [92]:
def sentence_gen():
    yield from iter(Sentence("The old man and sea"))
    yield from Sentence("Crime and Punishment")

In [93]:
sentence_gen()

<generator object sentence_gen at 0x112fda9b0>

In [94]:
list(sentence_gen())

['The', 'old', 'man', 'and', 'sea', 'Crime', 'and', 'Punishment']

In [135]:
def main_gen():
    while True:
        yield from averager()

In [136]:
main = main_gen()

In [137]:
main

<generator object main_gen at 0x1126ef640>

In [138]:
next(main)

In [139]:
main.send(30)

30.0

In [140]:
main.send(4)

17.0

In [141]:
main.send(60)

31.333333333333332

In [142]:
r = main.send(None)

In [145]:
def main_gen2():
    while True:
        yield from averager()
        yield from sentence_gen()

In [146]:
main = main_gen2()
next(main)

In [147]:
main.send(30)

30.0

In [148]:
next(main)

'The'

In [149]:
next(main)

'old'

In [150]:
main.send(40)

'man'

In [151]:
main.send(40)

'and'

In [154]:
next(main)

StopIteration: 

In [153]:
list(main)

KeyboardInterrupt: 

### Sample : 가상의 중1 학생 키와 몸무게 평균

In [7]:
from collections import namedtuple
Result = namedtuple('Result', 'count average')
# the subgenerator
def averager(): 
    total = 0.0
    count = 0 
    average = None 
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)


# the delegating generator
def grouper(results, key): 
    while True:
        results[key] = yield from averager()

# the client code, a.k.a. the caller
def main(data): 
    results = {}
    for key, values in data.items(): 
        group = grouper(results, key)
        print(f"{key}, {type(group)}, {id(group)}")
        next(group)
        for value in values:
            group.send(value) 
        #group.send(None) # important!
        print(results)  # uncomment to debug
    report(results) 
        
# output report
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print(f'{result.count:2} {group:5} averaging {result.average:.2f}{unit}')

data={ 
    'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}

main(data)

girls;kg, <class 'generator'>, 4456129952
{}
girls;m, <class 'generator'>, 4456130368
{}
boys;kg, <class 'generator'>, 4456129952
{}
boys;m, <class 'generator'>, 4456130368
{}


![averager](./images/group_averager.png)

In [3]:
data.items()

dict_items([('girls;kg', [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5]), ('girls;m', [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43]), ('boys;kg', [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3]), ('boys;m', [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46])])

In [4]:
for k, v in data.items():
    print(f"{k = }, { v =}")

k = 'girls;kg',  v =[40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5]
k = 'girls;m',  v =[1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43]
k = 'boys;kg',  v =[39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3]
k = 'boys;m',  v =[1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46]


# Async & Await 

In [44]:
def hello():
    print("Hello, Old-Fashioned World!")

hello()

Hello, Old-Fashioned World!


In [98]:
async def async_hello():
    print("Hello, Asynchronous World!")

async_hello()

<coroutine object async_hello at 0x1130abe80>

In [99]:
def gen_hello():
    yield 
    print("Hello, Generator World!")

gen_hello()

<generator object gen_hello at 0x1130abf40>

In [100]:
coro_hello = async_hello()
#next(coro_hello)
try:
    coro_hello.send(None)
except StopIteration as ex:
    print(ex.value)
    

Hello, Asynchronous World!
None


In [125]:
import types

@types.coroutine
def lazy_range(n):
    while n >= 0:
        yield n
        n -= 1
    return n


In [126]:
async def print_list():
    res = await lazy_range(5)
    print(f"{res = }")

In [127]:
coro_list = print_list()
try:
    coro_list.send(None)
    while True:
        print("Step", coro_list.send(None))
except StopIteration as ex:
    print(ex.value)

Step 4
Step 3
Step 2
Step 1
Step 0
res = -1
None


In [122]:
a = lazy_range(5)
a

<generator object lazy_range at 0x11353c880>

In [112]:
async def lazy_range(n):
    while n >= 0:
        yield n
        n -= 1


In [113]:
a = lazy_range(5)
a

<async_generator object lazy_range at 0x11353ca00>

In [116]:
async for i in a:
    print(i)

5
4
3
2
1
0


In [123]:
async def print_list():
    res = await lazy_range(5)
    print(f"{res = }")

In [124]:
coro_list = print_list()
try:
    coro_list.send(None)
    while True:
        print("Step", coro_list.send(None))
except StopIteration as ex:
    print(ex.value)
    

Step 4
Step 3
Step 2
Step 1
Step 0
res = -1
None


In [35]:
results = {}
key = 'girls;kg'
group = grouper(results, key)

In [36]:
group

<coroutine object grouper at 0x112d94d40>

In [39]:
group.send(None)

TypeError: object generator can't be used in 'await' expression

### Sentence를 Awaitable로 구현하기

In [150]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    def __init__(self, text):
        self.text = text

    def __await__(self):
        return (match.group() for match in RE_WORD.finditer(self.text))
        
    def __repr__(self):
        return f"Senetence ({reprlib.repr(self.text)})"

In [151]:
async def get_word():
    await Sentence("the old man and sea")

In [152]:
word = get_word()
word

<coroutine object get_word at 0x11353cdc0>

In [153]:
try:
    while True:
        print("Step", word.send(None))
except StopIteration as ex:
    print(ex.value)

Step the
Step old
Step man
Step and
Step sea
None


In [149]:
word.send(None)

RuntimeError: cannot reuse already awaited coroutine

### Averager를 awitable로 구현하기

In [199]:
from collections import namedtuple
Result = namedtuple('Result', 'count average')

import types

@types.coroutine
# the subgenerator
def averager(): 
    total = 0.0
    count = 0 
    average = None 
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)


In [200]:
await_avg = averager()
await_avg

<generator object averager at 0x1133ae890>

In [201]:
await_avg.send(None) # start

In [202]:
await_avg.send(30)
await_avg.send(40)
await_avg.send(50)

In [203]:
await_avg.send(None)

StopIteration: Result(count=3, average=40.0)

In [205]:
# the delegating generator
async def grouper(results, key): 
    while True:
        results[key] = await averager()

# the client code, a.k.a. the caller
def main(data): 
    results = {}
    for key, values in data.items(): 
        group = grouper(results, key)
        print(f"{key}, {type(group)}, {id(group)}")
        #next(group)
        group.send(None)
        for value in values:
            group.send(value) 
        group.send(None) # important!
        # print(results)  # uncomment to debug
    report(results) 
        
# output report
def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print(f'{result.count:2} {group:5} averaging {result.average:.2f}{unit}')

data={ 
    'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}

main(data)

girls;kg, <class 'coroutine'>, 4618413680
girls;m, <class 'coroutine'>, 4618596608
boys;kg, <class 'coroutine'>, 4618413680
boys;m, <class 'coroutine'>, 4618596608
 9 boys  averaging 40.42kg
 9 boys  averaging 1.39m
10 girls averaging 42.04kg
10 girls averaging 1.43m


In [206]:
group = grouper(results, key)
group

<coroutine object grouper at 0x1138453c0>

In [194]:
a = avg([10, 20, 30, 40])
a

<coroutine object avg at 0x1130c3680>

In [195]:
a.send(None)

In [196]:
a.send(10)
a.send(20)
a.send(30)
a.send(40)

In [197]:
a.send(None)

In [16]:

# the subgenerator
async def averager(): 
    total = 0.0
    count = 0 
    average = None 
    while True:
        term = await 1
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)


In [17]:
a = averager()

In [18]:
a.send(None)

TypeError: object int can't be used in 'await' expression